## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/542366/enhancing-rag-functionalities-using-tools-and-agents-in-langchain

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Installing and Importing Required Libraries

In [1]:
!pip install -U langchain
!pip install langchain-core
!pip install langchainhub
!pip install -qU langchain-openai
!pip install pypdf
!pip install faiss-cpu
!pip install --upgrade --quiet  wikipedia

In [2]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool
from langchain import hub
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

## Enhancing RAG with Tools and Agents

### Importing Wikipedia Tool

In [23]:
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
response = wikipedia_tool.run("What are large language models?")
print(response)

Page: Large language model
Summary: A large language model (LLM) is a computational model capable of language generation or other natural language processing tasks. As language models, LLMs acquire these abilities by learning statistical relationships from vast amounts of text during a self-supervised and semi-supervised training process.
The largest and most capable LLMs, as of August 2024, are artificial neural networks built with a decoder-only transformer-based architecture, which enables efficient processing and generation of large-scale text data. Modern models can be fine-tuned for specific tasks or can be guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained on.
Some notable LLMs are OpenAI's GPT series of models (e.g., GPT-3.5, GPT-4 and GPT-4o; used in ChatGPT and Microsoft Copilot), Google's Gemini (t

In [4]:
@tool
def WikipediaSearch(search_term: str):

    """
    Use this tool to search for wikipedia articles. 
    If a user asks to search the internet, you can search via this wikipedia tool.
    """
    

    result = wikipedia_tool.run(search_term)
    return result 
    

### Creating Retrieval Tool

In [5]:
openai_api_key = os.getenv('OPENAI_API_KEY')

loader = PyPDFLoader("https://web.archive.org/web/20170809122528id_/http://global-settlement.org/pub/The%20English%20Constitution%20-%20by%20Walter%20Bagehot.pdf")
docs = loader.load_and_split()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

In [6]:
query = """
"What is the difference between the house of lords and house of commons? How members are elected for both?"
"""
retriever.invoke(query)[:3]

[Document(metadata={'source': 'https://web.archive.org/web/20170809122528id_/http://global-settlement.org/pub/The%20English%20Constitution%20-%20by%20Walter%20Bagehot.pdf', 'page': 85}, page_content='It may be said that the House of Commons does not rule, it only elects the rulers. But there must be something  \nspecial about it to enable it to do that. Suppose the Cabinet were elected by a London club, what confusion  \nthere would be, what writing and answering! "Will you speak to So-and-So, and ask him to vote for my man?"  \nwould be heard on every side. How the wife of A. and the wife of B. would plot to confound the wife of C.  \nWhether the club elected under the dignified shadow of a queen, or without the shadow, would hardly matter  \nat all; if the substantial choice was in them, the confusion and intrigue would be there too. I propose to begin  \nthis paper by asking, not why the House of Commons governs well? but the fundamental—almost unasked  \nquestion—how the House of C

In [7]:
BritishParliamentSearch = create_retriever_tool(
    retriever,
    "british_parliament_search",
    "Use this tool tos earch for information about the british parliament, house of lords and house of common and any other related information.",
)

### Creating Tool Calling Agent

In [8]:
tools = [WikipediaSearch, BritishParliamentSearch]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

C:\Users\usman\anaconda3\Lib\site-packages\langchain\hub.py:120: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
llm = ChatOpenAI(model="gpt-4o",
                 temperature=0,
                 api_key=openai_api_key,
                )

In [10]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)
response = agent_executor.invoke({"input": "How many members are there in the House of Common?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `british_parliament_search` with `{'query': 'number of members in the House of Commons'}`


THE HOUSE OF COMMONS. ..................................................................................... 82
NO. VI. ...................................................................................................................... 101
ON CHANGES OF MINISTRY. .................................................................................... 101
NO. VII. ..................................................................................................................... 120
ITS SUPPOSED CHECKS AND BALANCES. ................................................................ 120
NO. VIII. .................................................................................................................... 136
THE PREREQUISITES OF CABINET GOVERNMENT, 
AND THE PECULIAR FORM WHICH THEY HAVE ASSUMED IN ENGLAND. .............136
NO. IX. ............

In [11]:
response = agent_executor.invoke({"input": "Who is the current president of United States?"})



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `{'search_term': 'President of the United States'}`


Page: President of the United States
Summary: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is one of the world's most powerful political figures and the leader of the world's only remaining superpower

In [12]:
print(response["output"])

The current president of the United States is Joe Biden. He is the 46th president and assumed office on January 20, 2021.


## Adding Memory to Agent Executor

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = False)

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [14]:
def generate_response(query):
    response = agent_with_chat_history.invoke(
    {"input": query},
    config={"configurable": {"session_id": "<foo>"}}
    )
    
    return response

In [18]:
query = "What is the difference between the house of lords and the house of commons?"
response = generate_response(query)
print(response["output"])

The House of Commons and the House of Lords are the two houses of the Parliament of the United Kingdom, each with distinct roles, compositions, and powers.

### House of Commons
- **Composition**: The House of Commons is an elected body consisting of 650 members known as Members of Parliament (MPs). These MPs are elected to represent constituencies by the first-past-the-post system.
- **Role**: The House of Commons is the primary legislative body. It is responsible for making and passing laws. The government is solely responsible to the House of Commons, and the Prime Minister stays in office only as long as they retain the confidence of a majority of the Commons.
- **Powers**: The House of Commons holds significant power, including the ability to pass legislation, control public spending, and scrutinize the government. Under the Parliament Acts of 1911 and 1949, the House of Lords' power to reject legislation was reduced to a delaying power.
- **History**: The House of Commons evolved

In [19]:
query = "Who is the current President of America?"
response = generate_response(query)
print(response["output"])

The current President of the United States is Joe Biden. He assumed office on January 20, 2021, and is the 46th president of the United States.


In [20]:
query = "And of France?"
response = generate_response(query)
print(response["output"])

The current President of France is Emmanuel Macron. He has been in office since May 14, 2017, and was re-elected for a second term in 2022.
